In [1]:
import os
import re
import json
import pandas as pd
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import time
import matplotlib.pyplot as plt
from googleapiclient.discovery import build

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

https://developers.google.com/youtube/v3/docs/comments?hl=fr#snippet.updatedAt

# On récupère l'id de la vidéo youtube

In [2]:
# https://www.youtube.com/watch?v=24Ob5JxhF4s
def url2id():
    video_url = input("Enter the video URL: ")
    pattern = r'^(https?:\/\/)?(www\.)?(youtube\.com\/(watch\?v=|embed\/|v\/)|youtu\.be\/)([\w-]{11})(\S+)?$'
    if re.match(pattern, video_url) :
        print("Valid YouTube URL")
    else:
        print("Error : invalid YouTube URL")

    video_id = video_url.split("v=")[-1]
    # print("Extracted video ID:", video_id)

    # Check if the video ID is valid
    if len(video_id) == 11:
        print("Valid video ID:", video_id)
        return video_id
    else:
        print("Error : invalid video ID. Please enter a valid YouTube video URL.")

id=url2id()

Valid YouTube URL
Valid video ID: 24Ob5JxhF4s


# On récupère les commentaires de la vidéo

In [27]:
from googleapiclient.discovery import build

with open('../secret_clientid.txt', 'r') as file:
        DEVELOPER_KEY = file.read()
# Configurez votre clé API et initialisez l'API YouTube
api_key = DEVELOPER_KEY
youtube = build("youtube", "v3", developerKey=api_key)

# Paramètres initiaux pour la requête
video_id = id
comments_data = []
next_page_token = None

while True:
    response = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100,
        order="time",
        textFormat="plainText",
        pageToken=next_page_token
    ).execute()

    # Ajouter les commentaires récupérés à la liste
    extraction_date = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    for item in response.get("items", []):
        comment_info = item["snippet"]["topLevelComment"]["snippet"]
        comments_data.append({
            "channelId": comment_info.get("channelId"),
            "videoId": comment_info.get("videoId"),	
            "author": comment_info.get("authorDisplayName"),
            "publishedAt": comment_info.get("publishedAt"),
            "comment": comment_info.get("textDisplay"),
            "extractedAt": extraction_date
        }) # ajouter un id

    # Vérifier s'il y a une page suivante
    next_page_token = response.get("nextPageToken")
    if not next_page_token:
        break
    time.sleep(1)

# Création du DataFrame à partir des données collectées
df = pd.DataFrame(comments_data)
print(df.head())
print(f"Total de commentaires récupérés : {len(comments_data)}")

# retirer le commentaire du youtubeur


                  channelId      videoId         author           publishedAt  \
0  UCK3inMNRNAVUleEbpDU1k2g  24Ob5JxhF4s       @SEBFRIT  2025-06-02T19:21:45Z   
1  UCK3inMNRNAVUleEbpDU1k2g  24Ob5JxhF4s   @guishoo4700  2025-06-30T18:45:12Z   
2  UCK3inMNRNAVUleEbpDU1k2g  24Ob5JxhF4s  @Remi-B-Goode  2025-06-30T16:35:49Z   
3  UCK3inMNRNAVUleEbpDU1k2g  24Ob5JxhF4s         @Guaph  2025-06-29T20:17:58Z   
4  UCK3inMNRNAVUleEbpDU1k2g  24Ob5JxhF4s    @rockman156  2025-06-29T19:05:12Z   

                                             comment          extractedAt  
0  Merci à Odoo le sponsor cette vidéo ! Pour déc...  2025-07-01 11:46:10  
1           Fait une vidéo sur Tyler the creator stp  2025-07-01 11:46:10  
2                                 Lol le disclaimer😂  2025-07-01 11:46:10  
3     où peut-on voir la série "Earn and Alfred" ???  2025-07-01 11:46:10  
4  C'est pas un re upload ? Impression de déjà vu...  2025-07-01 11:46:10  
Total de commentaires récupérés : 1731


In [28]:
display(df)

,channelId,videoId,author,publishedAt,comment,extractedAt
0,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@SEBFRIT,2025-06-02T19:21:45Z,Merci à Odoo le sponsor cette vidéo ! Pour déc...,2025-07-01 11:46:10
1,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@guishoo4700,2025-06-30T18:45:12Z,Fait une vidéo sur Tyler the creator stp,2025-07-01 11:46:10
2,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Remi-B-Goode,2025-06-30T16:35:49Z,Lol le disclaimer😂,2025-07-01 11:46:10
3,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Guaph,2025-06-29T20:17:58Z,"où peut-on voir la série ""Earn and Alfred"" ???",2025-07-01 11:46:10
4,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@rockman156,2025-06-29T19:05:12Z,C'est pas un re upload ? Impression de déjà vu...,2025-07-01 11:46:10
...,...,...,...,...,...,...
1726,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@sos3753,2025-06-02T17:49:39Z,Heyyy,2025-07-01 11:46:31
1727,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@bobonedegato,2025-06-02T17:49:38Z,Ça va me plaire ça,2025-07-01 11:46:31
1728,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@omasarys,2025-06-02T17:49:36Z,Ouii de nouvelles vidéos !!,2025-07-01 11:46:31
1729,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@globglobgabgalab4771,2025-06-02T17:49:35Z,Premier,2025-07-01 11:46:31


In [30]:
df['publishedAt']= pd.to_datetime(df['publishedAt'], format='ISO8601')  # Convertir la colonne 'publishedAt' en datetime
df['extractedAt'] = pd.to_datetime(df['extractedAt'], format='%Y-%m-%d %H:%M:%S')  # Convertir la colonne 'extractedAt' en datetime
df['channelId'] = df['channelId'].astype(str)  # Convertir la colonne 'channelId' en chaîne de caractères
df['videoId'] = df['videoId'].astype(str)  # Convertir la colonne 'videoId' en chaîne de caractères
df['author'] = df['author'].astype(str)  # Convertir la colonne 'author' en chaîne de caractères
df['comment'] = df['comment'].astype(str)  # Convertir la colonne 'comment' en chaîne de caractères


In [31]:
df.dtypes

channelId                   object
videoId                     object
author                      object
publishedAt    datetime64[ns, UTC]
comment                     object
extractedAt         datetime64[ns]
dtype: object

# Écrirture du fichier

In [32]:
df.to_csv('database_raw.csv', index=False)